In [ ]:
import requests
import shutil

from pathlib import Path
from tqdm import tqdm
from detector_utils import *

In [ ]:
base_path = Path('/Users/rimma_vakhreeva/Desktop/Test')
picture_url_3_sign = 'https://img1.wbstatic.net/big/new/{0:03d}0000/{0:03d}{1:04d}-{2}.jpg'
picture_url_4_sign = 'https://img1.wbstatic.net/big/new/{0:04d}0000/{0:04d}{1:04d}-{2}.jpg'

In [ ]:
def detect_person_head(image_filename):
    return [], []

In [ ]:
def is_valid_by_detections(person_bboxes, head_bboxes):
    def _filter_by_conf(bboxes, conf = 0.3):
        return [bbox for bbox in bboxes if bbox.meta > conf]

    person_bboxes = _filter_by_conf(person_bboxes, 0.2)
    head_bboxes = _filter_by_conf(head_bboxes, 0.3)
    
    # conf - уверенность модели в bounding box
    
    if len(person_bboxes) != 1 or len(head_bboxes) != 1: 
        return False
    #head и person должны быть одни на картинке
    
    person_bbox = person_bboxes[0]
    head_bbox = head_bboxes[0]
    person_bbox.clip_coords()
    head_bbox.clip_coords()
    
    x, y, _, _ = head_bbox.center_xywh()
    if y > 0.3:
        return False
    if x < 0.2 or x > 0.8:
        return False
    
    if person_bbox.width() > 0.65:
        return False
    x, y, _, _ = person_bbox.center_xywh()
    if y < 0.4 or y > 0.6:
        return False
    if x < 0.4 or x > 0.6:
        return False
    if 1. - person_bbox.height() > 0.04:
        return False
    
    area_rel = person_bbox.area() / head_bbox.area()
    if area_rel < 17. or area_rel > 35.:
        return False
    
    return True

In [ ]:
def save_image(url, output_filename):
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        return
    with open(output_filename, 'wb') as out_file:
        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, out_file)
        
def url_exists(url):
    return requests.get(url).status_code == 200

In [ ]:
# парсер wildberries
for category_id in tqdm(range(1, 9999)):
    four_sign = category_id > 999 # 4 знака в категории
    picture_url = picture_url_4_sign if four_sign else picture_url_3_sign
    
    if not url_exists(picture_url.format(category_id, 0, '1')):
        continue
        
    for product_id in tqdm(range(0, 9999)):
        for view_id in range(1, 10):
            url = picture_url.format(category_id, product_id, view_id)
            if not url_exists(url):
                break
                
            image_path = base_path / '{0:04d}_{1:03d}_{2}.jpg'.format(category_id, product_id, view_id)
            save_image(url, image_path)
            person_bboxes, head_bboxes = detect_person_head(image_path)
            if not is_valid_by_detections(person_bboxes, head_bboxes):
                base_path.unlink()

In [ ]:
images = read_all_images()

for image in images:
    person_bboxes, head_bboxes = detect_person_head(image)
    if not is_valid_by_detections(person_bboxes, head_bboxes):
        remove(image)